# Install `simex`

In [1]:
%%bash
#clone repo of `sipecam-metadata-extactor` in server
#git clone git@github.com:CONABIO/sipecam-metadata-extractor.git ~/
cd ../src
#install in editable mode
pip install -q -e .

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


# List of files to extract metadata

In [2]:
%%bash
dir_with_sipecam_data=/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/
list_of_files_to_extract_metadata --input_directory $dir_with_sipecam_data

## Check

In [3]:
%%bash
wc -l ~/sipecam_files_to_extract_metadata_from_*

29080 /root/sipecam_files_to_extract_metadata_from_11-2021.txt


# Extract metadata and ingest it

In [4]:
%%bash
head -n 1 ~/sipecam_files_to_extract_metadata_from_*

/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0049.JPG


In [5]:
%%bash
file_to_be_processed=$(head -n 1 ~/sipecam_files_to_extract_metadata_from_*)
extract_metadata_and_ingest_it --input_file "$file_to_be_processed"

## Check

In [6]:
%%bash
file_to_be_processed=$(head -n 1 ~/sipecam_files_to_extract_metadata_from_*)
dirname_file_to_be_processed=$(dirname "$file_to_be_processed")
temp_dir_for_logs=$(find "$dirname_file_to_be_processed" -name 'temp*')
basename_file_to_be_processed=$(basename "$file_to_be_processed")
cat "$temp_dir_for_logs/logs_$basename_file_to_be_processed"

2021-11-30 12:30:01,782 [INFO ]  extract metadata and ingest it
2021-11-30 12:30:01,784 [INFO ]  JPG
